### Predicting Wind Turbine Capacity using Regression

Goal: Using the characteristics of wind turbines from 1983 to 2021, I want to create a regression model that will be able to accurately predict the electrical generation capacity of a wind turbine (kW).

# Import libraries
We will be using pandas, numpy and scikit-learn for pre-processing and developing model. Bokeh will help with visualization of model and data.

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, RidgeCV
from sklearn.pipeline import make_pipeline
from bokeh.plotting import figure, show, output_notebook
output_notebook()

Loading BokehJS ...

# Pre-processing

### Import and Read Data

In [3]:
data = pd.read_csv('wind_turbines.csv')
data

,Site.State,Site.County,Year,Turbine.Capacity,Turbine.Hub_Height,Turbine.Rotor_Diameter,Turbine.Swept_Area,Turbine.Total_Height,Project.Capacity,Project.Number_Turbines,Site.Latitude,Site.Longitude
0,IA,Story County,2017,3000,87.5,125.0,12271.85,150.0,30.000,10,-93.518082,42.013630
1,IA,Hardin County,2017,3000,87.5,125.0,12271.85,150.0,30.000,10,-93.367798,42.497940
2,IA,Story County,2017,3000,87.5,125.0,12271.85,150.0,30.000,10,-93.513710,42.019119
3,IA,Story County,2017,3000,87.5,125.0,12271.85,150.0,30.000,10,-93.523651,42.006813
4,IA,Story County,2017,3000,87.5,125.0,12271.85,150.0,30.000,10,-93.632835,41.882477
...,...,...,...,...,...,...,...,...,...,...,...,...
63956,TX,El Paso County,2015,1700,80.0,100.0,7853.98,130.1,3.400,2,-106.405434,31.788124
63957,TX,El Paso County,2015,1700,80.0,100.0,7853.98,130.1,3.400,2,-106.405670,31.788097
63958,NY,Erie County,2016,100,37.0,21.0,346.36,47.5,0.200,2,-78.931122,42.977005
63959,AK,Northwest Arctic Borough,2003,65,30.5,15.0,176.71,38.0,0.975,15,-162.551575,66.837898


In [4]:
data.describe()

,Year,Turbine.Capacity,Turbine.Hub_Height,Turbine.Rotor_Diameter,Turbine.Swept_Area,Turbine.Total_Height,Project.Capacity,Project.Number_Turbines,Site.Latitude,Site.Longitude
count,63961.000000,63961.000000,63961.000000,63961.000000,63961.000000,63961.000000,63961.000000,63961.000000,63961.000000,63961.000000
mean,2011.768593,1945.300714,80.265244,94.682148,7472.673751,127.634646,171.037738,96.923172,-99.688416,38.592440
std,6.444849,689.733922,12.466083,23.449221,3267.170143,22.946809,102.126002,69.180367,11.005005,5.487171
min,1983.000000,50.000000,22.800000,14.000000,153.940000,30.400000,0.050000,1.000000,-171.713074,13.389381
25%,2008.000000,1500.000000,80.000000,82.000000,5281.020000,121.000000,100.000000,56.000000,-102.597290,34.252995
50%,2012.000000,2000.000000,80.000000,97.000000,7389.810000,130.100000,160.000000,85.000000,-99.329704,39.316921
75%,2017.000000,2300.000000,85.000000,110.000000,9503.320000,144.500000,211.220000,120.000000,-94.947594,42.991066
max,2021.000000,6000.000000,131.000000,155.000000,18869.190000,199.600000,525.020000,460.000000,144.722656,66.839905


Large differences in magnitude with each column, so will need to do some scaling

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63961 entries, 0 to 63960
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Site.State               63961 non-null  object 
 1   Site.County              63961 non-null  object 
 2   Year                     63961 non-null  int64  
 3   Turbine.Capacity         63961 non-null  int64  
 4   Turbine.Hub_Height       63961 non-null  float64
 5   Turbine.Rotor_Diameter   63961 non-null  float64
 6   Turbine.Swept_Area       63961 non-null  float64
 7   Turbine.Total_Height     63961 non-null  float64
 8   Project.Capacity         63961 non-null  float64
 9   Project.Number_Turbines  63961 non-null  int64  
 10  Site.Latitude            63961 non-null  float64
 11  Site.Longitude           63961 non-null  float64
dtypes: float64(7), int64(3), object(2)
memory usage: 5.9+ MB


We can see that none of the data is non-null, which makes pre-processing simplier

### Rename and organize data

In [6]:
y = data['Turbine.Capacity']
X = data.drop(columns=['Turbine.Capacity', 'Project.Capacity', 'Project.Number_Turbines', 'Site.State', 'Site.County'])
X = X.rename(columns={'Year' : 'year', 'Turbine.Hub_Height': 'hub_height', 'Turbine.Rotor_Diameter': 'rotor_diameter', 'Turbine.Swept_Area': 'swept_area', 'Turbine.Total_Height': 'total_height', 'Site.Latitude' : 'latitude', 'Site.Longitude' : 'longitude'})
X

,year,hub_height,rotor_diameter,swept_area,total_height,latitude,longitude
0,2017,87.5,125.0,12271.85,150.0,-93.518082,42.013630
1,2017,87.5,125.0,12271.85,150.0,-93.367798,42.497940
2,2017,87.5,125.0,12271.85,150.0,-93.513710,42.019119
3,2017,87.5,125.0,12271.85,150.0,-93.523651,42.006813
4,2017,87.5,125.0,12271.85,150.0,-93.632835,41.882477
...,...,...,...,...,...,...,...
63956,2015,80.0,100.0,7853.98,130.1,-106.405434,31.788124
63957,2015,80.0,100.0,7853.98,130.1,-106.405670,31.788097
63958,2016,37.0,21.0,346.36,47.5,-78.931122,42.977005
63959,2003,30.5,15.0,176.71,38.0,-162.551575,66.837898


### Convert nominal data to numeric data

In [7]:
state = pd.get_dummies(data['Site.State'], drop_first=True)
county = pd.get_dummies(data['Site.County'], drop_first=True)
X = X.join(state)
X = X.join(county)
X

,year,hub_height,rotor_diameter,swept_area,total_height,latitude,longitude,AZ,CA,CO,...,Woodward County,Worcester County,Worth County,Wright County,Wyandot County,Wyoming County,Yolo County,Young County,Yuma County,Zapata County
0,2017,87.5,125.0,12271.85,150.0,-93.518082,42.013630,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,2017,87.5,125.0,12271.85,150.0,-93.367798,42.497940,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,2017,87.5,125.0,12271.85,150.0,-93.513710,42.019119,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,2017,87.5,125.0,12271.85,150.0,-93.523651,42.006813,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,2017,87.5,125.0,12271.85,150.0,-93.632835,41.882477,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63956,2015,80.0,100.0,7853.98,130.1,-106.405434,31.788124,False,False,False,...,False,False,False,False,False,False,False,False,False,False
63957,2015,80.0,100.0,7853.98,130.1,-106.405670,31.788097,False,False,False,...,False,False,False,False,False,False,False,False,False,False
63958,2016,37.0,21.0,346.36,47.5,-78.931122,42.977005,False,False,False,...,False,False,False,False,False,False,False,False,False,False
63959,2003,30.5,15.0,176.71,38.0,-162.551575,66.837898,False,False,False,...,False,False,False,False,False,False,False,False,False,False


### Split into training and testing data

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

### Principal Component Analysis

We can take each of the columns of the data and turn them into one component to help visualize the general trend and relationship of the data.

In [9]:
cols = X.columns

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

pca = PCA(n_components=1)
X_train_pca = pca.fit_transform(X_train_scaled)

train_plot_pca = figure(title='PCA on Numeric Inputs vs Turbine Capacity (kW)', x_axis_label='Principal Component', y_axis_label='Turbine Capacity')
train_plot_pca.circle(x=X_train_pca.ravel(), y=y_train, radius=0.02)
show(train_plot_pca)

From this, we can see a reverse logarithmic/polynomial relationship that we should look into

In [10]:
train_plot = figure(title='Training Data', x_axis_label='Rotor Diameter', y_axis_label='Turbine Capacity (kW)')
train_plot.circle(x=X_train['rotor_diameter'], y=y_train, radius=1, color="blue")
show(train_plot)

By viewing the individual relationship with the rotor diameter, where we see a linear/square function of the data.

# Model Creation

We can test a linear, polynomial and logarithmic regression model

## Linear Model

For the linear model, we can create a model to represent the relationship between the turbine capacity and the rotor diamter

In [11]:
linear_model = make_pipeline(StandardScaler(), LinearRegression())
linear_model.fit(X_train['rotor_diameter'].to_frame(), y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('linearregression', LinearRegression())])

Compare predictions between model and actual value

In [12]:
train_predictions_linear = linear_model.predict(X_train['rotor_diameter'].to_frame())
test_predictions_linear = linear_model.predict(X_test['rotor_diameter'].to_frame())

In [13]:
from bokeh.layouts import row

def visualize_model(X_train, X_test, train_preds, test_preds):
    train_plot_cubic = figure(title='Train Data', x_axis_label='Rotor Diameter', y_axis_label='Turbine Capacity (kW)')
    test_plot_cubic = figure(title='Test Data', x_axis_label='Rotor Diameter', y_axis_label='Turbine Capacity (kW)')

    train_plot_cubic.circle(x=X_train, y=y_train, radius=1, color='blue', legend_label='Actual')
    train_plot_cubic.circle(x=X_train, y=train_preds, radius=1, color='red', legend_label='Predicted')

    test_plot_cubic.circle(x=X_test, y=y_test, color='blue', radius=1, legend_label='Actual')
    test_plot_cubic.circle(x=X_test, y=test_preds, color='red', radius=1, legend_label='Predicted')

    show(row(train_plot_cubic,test_plot_cubic))

By plotting the data, we can see that the linear model has a fairly good accuracy when predicting turbine capacity using just the rotor diameter

In [14]:
visualize_model(X_train['rotor_diameter'], X_test['rotor_diameter'], train_predictions_linear, test_predictions_linear)

test_score = linear_model.score(X_test['rotor_diameter'].to_frame(), y_test)
train_score = linear_model.score(X_train['rotor_diameter'].to_frame(), y_train)
print(f'Test score: {test_score}')
print(f'Train score: {train_score}')

Test score: 0.7767480984833155
Train score: 0.7802849371252115


## Polynomial Model
We can create a cubic regression model, since the data follows a similar trend to this function through PCA. First, we can create our polynomial features for our numeric data to the degree of 3 (Cubic).

In [15]:
cubic = PolynomialFeatures(degree=3, include_bias=False)

# Remove nominal features from training and testing
X_train_numeric = X_train[['Year', 'hub_height', 'rotor_diameter', 'swept_area', 'total_height', 'latitude', 'longitude']]
X_test_numeric = X_test[['Year', 'hub_height', 'rotor_diameter', 'swept_area', 'total_height', 'latitude', 'longitude']]

# Fit a cubic model using numeric data
X_train_cubic = cubic.fit_transform(X_train_numeric)
X_test_cubic = cubic.fit_transform(X_test_numeric)

X_test_cubic

KeyError: "['Year'] not in index"

Combine cubic transformed data and with nominal data

In [ ]:
X_train_nominal = X_train.drop(columns=['Year', 'hub_height', 'rotor_diameter', 'swept_area', 'total_height', 'latitude', 'longitude'])
X_test_nominal = X_test.drop(columns=['Year', 'hub_height', 'rotor_diameter', 'swept_area', 'total_height', 'latitude', 'longitude'])
X_train_cubic_full = np.hstack([X_train_cubic, X_train_nominal])
X_test_cubic_full = np.hstack([X_test_cubic, X_test_nominal])

Create pipeline to scale and fit data

In [ ]:
cubic_model = make_pipeline(StandardScaler(), RidgeCV())
cubic_model.fit(X_train_cubic_full, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()), ('ridgecv', RidgeCV())])

View predictions using the cubic model

In [ ]:
train_predictions_cubic = cubic_model.predict(X_train_cubic_full)
test_predictions_cubic = cubic_model.predict(X_test_cubic_full)
test_predictions_cubic

array([1627.81954953, 2591.34954574, 2334.01473892, ..., 1617.98084643,
       2054.39040429,  746.87425378])

We can see a fairly high score using the cubic model, and by comparing the differences in the predicted and actual values there is only around a 10 percent difference, which is fairly accurate.

In [ ]:
cubic_model.score(X_test_cubic_full, y_test)

0.9164124310526054

In [ ]:
def displayDifference(pred, actual):
    df = pd.DataFrame({"Predicted": pred, "Actual": actual})
    df['% Difference'] = (abs(df['Predicted']-df['Actual'])/df['Actual'])*100

    print("Percentage Difference between Predicted and Actual Values (Cubic Model)")
    print(df.head())
    print("\nMean % Difference between Predicted and Actual Values: " + str(df['% Difference'].mean()) +"%")

In [ ]:
displayDifference(test_predictions_cubic, y_test)

Percentage Difference between Predicted and Actual Values (Cubic Model)
         Predicted  Actual  % Difference
61926  1627.819550    1650      1.344270
31818  2591.349546    2625      1.281922
25385  2334.014739    2500      6.639410
49576   974.322985    1000      2.567702
12172  2251.741526    2300      2.098195

Mean % Difference between Predicted and Actual Values: 8.768379001972097%


## Reciprocal Logarithmic Model

We can also try a reciprocal logarithmic model as it also follows a similar trend to the PCA relationship funtion we graphed

In [ ]:
# Create function to add 1/log(x) features
def addReciprocalLogFeatures(numeric):
    log_feats = numeric.copy()
    valid = (log_feats != 1) & (log_feats > 0)
    log_feats[valid] = np.log(log_feats[valid]) / np.log(10)
    log_feats[log_feats <= 0] = 1e-10
    rec_log_feats = 1 / log_feats
    return np.hstack([numeric, rec_log_feats, numeric * rec_log_feats])

We can follow a similar process with the cubic model, this time by creating our function to create our own reciprocal log features for all of our numeric data.

In [ ]:
X_train_rl = addReciprocalLogFeatures(X_train_numeric)
X_test_rl = addReciprocalLogFeatures(X_test_numeric)

C:\Users\wangb\AppData\Local\Temp\ipykernel_20628\625098222.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[[3.30405947]
 [3.3047059 ]
 [3.30276371]
 ...
 [3.30535137]
 [3.30168095]
 [3.30513632]]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  log_feats[valid] = np.log(log_feats[valid]) / np.log(10)
C:\Users\wangb\AppData\Local\Temp\ipykernel_20628\625098222.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[[3.30297994]
 [3.30492116]
 [3.30492116]
 ...
 [3.30276371]
 [3.30319606]
 [3.30081279]]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  log_feats[valid] = np.log(log_feats[valid]) / np.log(10)


We can combine the nominal and numeric data

In [ ]:
X_train_rl_full = np.hstack([X_train_rl, X_train_nominal])
X_test_rl_full = np.hstack([X_test_rl, X_test_nominal])

Create a pipeline to scale and fit data

In [ ]:
rl_model = make_pipeline(StandardScaler(), RidgeCV())
rl_model.fit(X_train_rl_full, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()), ('ridgecv', RidgeCV())])

View predictions made by model

In [ ]:
train_predictions_rl = rl_model.predict(X_train_rl_full)
test_predictions_rl = rl_model.predict(X_test_rl_full)
test_predictions_rl

array([1603.79020453, 2600.00346419, 2452.19082734, ..., 1617.21205921,
       2061.4258264 ,  756.24869151])

We can also see a very high accuracy score and small percent difference when using the RL model, which supports the idea that this model is well suited for predicting the turbine capacity as well

In [ ]:
rl_model.score(X_test_rl_full, y_test)

0.9100111394939737

In [ ]:
displayDifference(test_predictions_rl, y_test)

Percentage Difference between Predicted and Actual Values (Cubic Model)
         Predicted  Actual  % Difference
61926  1603.790205    1650      2.800594
31818  2600.003464    2625      0.952249
25385  2452.190827    2500      1.912367
49576  1008.624635    1000      0.862464
12172  2254.950893    2300      1.958657

Mean % Difference between Predicted and Actual Values: 9.742981653673311%
